#  Inférence de l'effet - Stratégie Multilabels - Approche deep Learning
Dans ce Notebook, nous cosntruisons un modèle qui permet d'inférer l'EFFET à partir de la classification de l'incident et des données textuelles en ce basant sur des reseau récurents commes GRU/LSTM etc.

En effet, ces approches ont montré des réultats très encourageant et nous voulons explorer cette direction pour peut être faire des réceau recurent notre modèle par défault.

Nous considérons ce problème comme un problème de classification multiclasses et multilabels. En effet, il y a plusieurs effets possibles et un incidents peut entrainer plusieurs effets.

Dans ce note book nous nous posons les questions suivantes : 
- Quel est l'impact du drop out ?
- Rajouter des couches augmentent-ils les performaces ?
- L'utilisation de réseaux bidirectionnel est-elle pertinente ?
- Une couche d'attention est-elle utile ?
- Attention is all we need, really ?
- Utilisation des embeddings 
- Concaténation des modèles sur différentes entrées ?

In [425]:
import pandas as pd
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense,LSTM,Embedding,SpatialDropout1D, Bidirectional,SimpleRNN,Input, concatenate, Reshape
import tensorflow 

from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer
from keras.layers import Concatenate, GlobalMaxPool1D, Dropout
from tensorflow.keras.layers import Attention

tensorflow.random.set_seed(1234)

## 1. Chargement des données

In [336]:
mlb = MultiLabelBinarizer()

train = pd.read_pickle('./data_split/train.pkl')
# Pour faire un modèle sans le 
#train = train[~train['TEF_ID'].map(lambda x : 106 in x)]
X_train = train[['FABRICANT','CLASSIFICATION','DESCRIPTION_INCIDENT','ETAT_PATIENT']]
y_train = mlb.fit_transform(train['TEF_ID'])
test =  pd.read_pickle('./data_split/test.pkl')
#test = test[~test['TEF_ID'].map(lambda x : k in x)]
X_test = test[['FABRICANT','CLASSIFICATION','DESCRIPTION_INCIDENT','ETAT_PATIENT']]
y_test = mlb.transform(test['TEF_ID'])


X_train_dgs = np.load('results/dgs_camenbert_train_vec.npy')
X_test_dgs =np.load('results/dgs_camenbert_test_vec.npy')



df_effets = pd.read_csv("data/ref_MRV/referentiel_dispositif_effets_connus.csv",delimiter=';',encoding='ISO-8859-1')
df_dys = pd.read_csv("data/ref_MRV/referentiel_dispositif_dysfonctionnement.csv",delimiter=';',encoding='ISO-8859-1')

## 2.1 LSTM et TFIDF, une première baseline

In [337]:
%%time
from sklearn.compose import ColumnTransformer
from sklearn.feature_extraction.text import TfidfVectorizer,HashingVectorizer
import spacy
nlp =spacy.load('fr')
from spacy.lang.fr.stop_words import STOP_WORDS

preprocess = ColumnTransformer(
    [('description_tfidf',TfidfVectorizer(sublinear_tf=True, min_df=3,
                            ngram_range=(1, 1),
                            stop_words=STOP_WORDS,
                            max_features = 10000,norm = 'l2'), 'DESCRIPTION_INCIDENT'),
     
     ('etat_pat_tfidf', TfidfVectorizer(sublinear_tf=True, min_df=3,ngram_range=(1, 1),
                                       stop_words=STOP_WORDS,
                                       max_features = 10000,norm = 'l2'), 'ETAT_PATIENT'),
     
     ('fabricant_tfidf',TfidfVectorizer(sublinear_tf=True, min_df=3,
                            ngram_range=(1, 1),
                            stop_words=STOP_WORDS,
                            max_features = 5000,norm = 'l2'), 'FABRICANT')
     ],
    
    remainder='passthrough')

X_train_, X_test_ =preprocess.fit_transform(X_train),preprocess.transform(X_test)

from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=1000)
X_train_ = svd.fit_transform(X_train_)
X_test_ = svd.transform(X_test_)


/home/user1/DGS-env/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:386: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['qu', 'quelqu'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
/home/user1/DGS-env/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:386: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['qu', 'quelqu'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
/home/user1/DGS-env/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:386: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['qu', 'quelqu'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


CPU times: user 2min 55s, sys: 2min 52s, total: 5min 47s
Wall time: 1min 6s


In [338]:
X_train_ = np.reshape(X_train_, (X_train_.shape[0], 1, X_train_.shape[1]))
X_test_ = np.reshape(X_test_, (X_test_.shape[0], 1, X_test_.shape[1]))

In [318]:
model = Sequential()
model.add(LSTM(200))
model.add(Dense(y_train.shape[1], activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

epochs = 10
batch_size = 32

history = model.fit(X_train_, y_train, epochs=epochs, batch_size=batch_size,validation_split=0.2)

score,cat_acc = model.evaluate(X_test_,y_test)

y_pred = model.predict(X_test_)

print('loss : ', score)
print('categorical accuracy: ',cat_acc)

print('####################################')

thresholds = [0.01,0.04,0.06,0.08,0.1,0.12,0.14,0.16,0.2,0.25,0.3,0.35,0.4,0.5,0.6,0.7]
for val in thresholds:
    print("For threshold: ", val)
    pred=y_pred.copy()
  
    pred[pred>=val]=1
    pred[pred<val]=0
  
    precision = precision_score(y_test, pred, average='samples')
    recall = recall_score(y_test, pred, average='samples')
    f1 = f1_score(y_test, pred, average='samples')
   
    print("Samples-average quality numbers")
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

Train on 21059 samples, validate on 5265 samples
Epoch 1/10
21059/21059 [==============================] - 11s 531us/step - loss: 0.0205 - categorical_accuracy: 0.5677 - val_loss: 0.0137 - val_categorical_accuracy: 0.5991
Epoch 2/10
21059/21059 [==============================] - 10s 491us/step - loss: 0.0149 - categorical_accuracy: 0.6200 - val_loss: 0.0125 - val_categorical_accuracy: 0.6380
Epoch 3/10
21059/21059 [==============================] - 10s 480us/step - loss: 0.0136 - categorical_accuracy: 0.6497 - val_loss: 0.0120 - val_categorical_accuracy: 0.6422
Epoch 4/10
21059/21059 [==============================] - 10s 460us/step - loss: 0.0127 - categorical_accuracy: 0.6651 - val_loss: 0.0118 - val_categorical_accuracy: 0.6424
Epoch 5/10
21059/21059 [==============================] - 7s 356us/step - loss: 0.0121 - categorical_accuracy: 0.6727 - val_loss: 0.0116 - val_categorical_accuracy: 0.6477
Epoch 6/10
21059/21059 [==============================] - 9s 449us/step - loss: 0.0116 

/home/user1/DGS-env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Samples-average quality numbers
Precision: 0.6239, Recall: 0.7872, F1-measure: 0.6618
For threshold:  0.1
Samples-average quality numbers
Precision: 0.6483, Recall: 0.7660, F1-measure: 0.6700
For threshold:  0.12
Samples-average quality numbers
Precision: 0.6640, Recall: 0.7496, F1-measure: 0.6730
For threshold:  0.14
Samples-average quality numbers
Precision: 0.6724, Recall: 0.7332, F1-measure: 0.6718
For threshold:  0.16
Samples-average quality numbers
Precision: 0.6736, Recall: 0.7159, F1-measure: 0.6673
For threshold:  0.2
Samples-average quality numbers
Precision: 0.6647, Recall: 0.6844, F1-measure: 0.6540
For threshold:  0.25
Samples-average quality numbers
Precision: 0.6487, Recall: 0.6560, F1-measure: 0.6386
For threshold:  0.3
Samples-average quality numbers
Precision: 0.6376, Recall: 0.6332, F1-measure: 0.6258
For threshold:  0.35
Samples-average quality numbers
Precision: 0.6224, Recall: 0.6096, F1-measure: 0.6090
For threshold:  0.4
Samples-average quality numbers
Precision

## Commentaires : 

Nous obtenons un score de base line de 0.6730. Nous allons essayer d'améliorer ce score avec difféntes approches : 
- Remplacer le tfidf par un embedding entrainé par le modèle 
    - Embedding simple
    - Embedding par colonnes
    
- Essayer un modèle biLSTM ou GRU ou bi GRU
- Essayer de rajouter une couche d'attention dans Keras


# 2.2 Comment performe un reseau de BiLSTM  ?


In [319]:
from keras.layers import Bidirectional

In [320]:
model = Sequential()
model.add(Bidirectional(LSTM(200, dropout=0.1)))
model.add(Dense(y_train.shape[1], activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

epochs = 5
batch_size = 32

history = model.fit(X_train_, y_train, epochs=epochs, batch_size=batch_size,validation_split=0.2)

score,cat_acc = model.evaluate(X_test_,y_test)

y_pred = model.predict(X_test_)

print('loss : ', score)
print('categorical accuracy: ',cat_acc)

print('####################################')

thresholds = [0.01,0.04,0.06,0.08,0.1,0.12,0.14,0.16,0.2,0.25,0.3,0.35,0.4,0.5,0.6,0.7]
for val in thresholds:
    print("For threshold: ", val)
    pred=y_pred.copy()
  
    pred[pred>=val]=1
    pred[pred<val]=0
  
    precision = precision_score(y_test, pred, average='samples')
    recall = recall_score(y_test, pred, average='samples')
    f1 = f1_score(y_test, pred, average='samples')
   
    print("Samples-average quality numbers")
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

Train on 21059 samples, validate on 5265 samples
Epoch 1/5
21059/21059 [==============================] - 17s 806us/step - loss: 0.0197 - categorical_accuracy: 0.5722 - val_loss: 0.0133 - val_categorical_accuracy: 0.6122
Epoch 2/5
21059/21059 [==============================] - 15s 711us/step - loss: 0.0146 - categorical_accuracy: 0.6302 - val_loss: 0.0123 - val_categorical_accuracy: 0.6414
Epoch 3/5
21059/21059 [==============================] - 15s 729us/step - loss: 0.0133 - categorical_accuracy: 0.6547 - val_loss: 0.0118 - val_categorical_accuracy: 0.6427
Epoch 4/5
21059/21059 [==============================] - 15s 691us/step - loss: 0.0125 - categorical_accuracy: 0.6640 - val_loss: 0.0115 - val_categorical_accuracy: 0.6475
Epoch 5/5
6580/6580 [==============================] - 1s 196us/step
loss :  0.012372263854986628
categorical accuracy:  0.6585106253623962
####################################
For threshold:  0.01
Samples-average quality numbers
Precision: 0.3066, Recall: 0.9161

## Commentaire ?

L'augementation de la performance est très minime : 0.6733 ?
Nous remarquons que notre modèle sur apprend assez rapidement, nous allons essayer d'ajouter une couche de dropout pour limiter ce sur apprentissage.
## Si on ajoute du dropout ?

In [321]:
model = Sequential()
model.add(Bidirectional(LSTM(200, return_sequences=True, dropout=0.1)))
model.add(GlobalMaxPool1D())
model.add(Dense(100, activation="elu"))
model.add(Dropout(0.1))
model.add( Dense(y_train.shape[1], activation="softmax"))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])


epochs = 10
batch_size = 32

history = model.fit(X_train_, y_train, epochs=epochs, batch_size=batch_size,validation_split=0.2)

score,cat_acc = model.evaluate(X_test_,y_test)

y_pred = model.predict(X_test_)

print('loss : ', score)
print('categorical accuracy: ',cat_acc)

print('####################################')

thresholds = [0.01,0.04,0.06,0.08,0.1,0.12,0.14,0.16,0.2,0.25,0.3,0.35,0.4,0.5,0.6,0.7]
for val in thresholds:
    print("For threshold: ", val)
    pred=y_pred.copy()
  
    pred[pred>=val]=1
    pred[pred<val]=0
  
    precision = precision_score(y_test, pred, average='samples')
    recall = recall_score(y_test, pred, average='samples')
    f1 = f1_score(y_test, pred, average='samples')
   
    print("Samples-average quality numbers")
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

Train on 21059 samples, validate on 5265 samples
Epoch 1/10
21059/21059 [==============================] - 15s 732us/step - loss: 0.0186 - categorical_accuracy: 0.5732 - val_loss: 0.0136 - val_categorical_accuracy: 0.6118
Epoch 2/10
21059/21059 [==============================] - 14s 669us/step - loss: 0.0148 - categorical_accuracy: 0.6201 - val_loss: 0.0125 - val_categorical_accuracy: 0.6298
Epoch 3/10
21059/21059 [==============================] - 15s 696us/step - loss: 0.0137 - categorical_accuracy: 0.6484 - val_loss: 0.0122 - val_categorical_accuracy: 0.6349
Epoch 4/10
21059/21059 [==============================] - 14s 673us/step - loss: 0.0130 - categorical_accuracy: 0.6556 - val_loss: 0.0119 - val_categorical_accuracy: 0.6397
Epoch 5/10
21059/21059 [==============================] - 14s 656us/step - loss: 0.0125 - categorical_accuracy: 0.6635 - val_loss: 0.0119 - val_categorical_accuracy: 0.6410
Epoch 6/10
21059/21059 [==============================] - 13s 637us/step - loss: 0.012

## Commentaire ?

Les performances changent très peu,  nous allons essayer d'augementer la profondeur de notre reseau.

## Si on ajoute plusieurs couches ?

In [323]:
from tensorflow.keras import regularizers
reg = regularizers.l2(1e-4)
model = Sequential()
model.add(Bidirectional(LSTM(256,  return_sequences=True, dropout=0.1)))
model.add((SimpleRNN(128,return_sequences=True)))
model.add(GlobalMaxPool1D())
model.add(Dropout(0.1))
model.add(Dense(y_train.shape[1], activation="softmax"))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])


epochs = 10
batch_size = 32

history = model.fit(X_train_, y_train, epochs=epochs, batch_size=batch_size,validation_split=0.2)

score,cat_acc = model.evaluate(X_test_,y_test)

y_pred = model.predict(X_test_)

print('loss : ', score)
print('categorical accuracy: ',cat_acc)

print('####################################')

thresholds = [0.01,0.04,0.06,0.08,0.1,0.12,0.14,0.16,0.2,0.25,0.3,0.35,0.4,0.5,0.6,0.7]
for val in thresholds:
    print("For threshold: ", val)
    pred=y_pred.copy()
  
    pred[pred>=val]=1
    pred[pred<val]=0
  
    precision = precision_score(y_test, pred, average='samples')
    recall = recall_score(y_test, pred, average='samples')
    f1 = f1_score(y_test, pred, average='samples')
   
    print("Samples-average quality numbers")
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

Train on 21059 samples, validate on 5265 samples
Epoch 1/10
21059/21059 [==============================] - 23s 1ms/step - loss: 0.0181 - categorical_accuracy: 0.5780 - val_loss: 0.0132 - val_categorical_accuracy: 0.6114
Epoch 2/10
21059/21059 [==============================] - 23s 1ms/step - loss: 0.0144 - categorical_accuracy: 0.6289 - val_loss: 0.0122 - val_categorical_accuracy: 0.6452
Epoch 3/10
21059/21059 [==============================] - 22s 1ms/step - loss: 0.0134 - categorical_accuracy: 0.6505 - val_loss: 0.0120 - val_categorical_accuracy: 0.6380
Epoch 4/10
21059/21059 [==============================] - 23s 1ms/step - loss: 0.0128 - categorical_accuracy: 0.6605 - val_loss: 0.0117 - val_categorical_accuracy: 0.6391
Epoch 5/10
21059/21059 [==============================] - 21s 1ms/step - loss: 0.0123 - categorical_accuracy: 0.6672 - val_loss: 0.0117 - val_categorical_accuracy: 0.6416
Epoch 6/10
21059/21059 [==============================] - 23s 1ms/step - loss: 0.0119 - categori

/home/user1/DGS-env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Samples-average quality numbers
Precision: 0.6510, Recall: 0.7618, F1-measure: 0.6704
For threshold:  0.12
Samples-average quality numbers
Precision: 0.6659, Recall: 0.7449, F1-measure: 0.6726
For threshold:  0.14
Samples-average quality numbers
Precision: 0.6761, Recall: 0.7289, F1-measure: 0.6719
For threshold:  0.16
Samples-average quality numbers
Precision: 0.6820, Recall: 0.7161, F1-measure: 0.6708
For threshold:  0.2
Samples-average quality numbers
Precision: 0.6804, Recall: 0.6945, F1-measure: 0.6641
For threshold:  0.25
Samples-average quality numbers
Precision: 0.6659, Recall: 0.6664, F1-measure: 0.6498
For threshold:  0.3
Samples-average quality numbers
Precision: 0.6514, Recall: 0.6436, F1-measure: 0.6370
For threshold:  0.35
Samples-average quality numbers
Precision: 0.6343, Recall: 0.6231, F1-measure: 0.6216
For threshold:  0.4
Samples-average quality numbers
Precision: 0.6186, Recall: 0.6024, F1-measure: 0.6052
For threshold:  0.5
Samples-average quality numbers
Precision

### Commentaires : 

Encore une fois, le modèle semble avoir atteint un plafond de verre qui ne soit pas dépassable ni par la prodondeur, ni par l'ajout de droptout. Nous allons essayer d'ajouter une couche d'attention.

## Une couche d'attention ?

In [354]:
from keras_self_attention import SeqSelfAttention

model = Sequential()
model.add(LSTM(200, return_sequences=True))
model.add(SeqSelfAttention(attention_activation='sigmoid'))
model.add(keras.layers.Flatten(name='Flatten'))
model.add(Dense(y_train.shape[1], activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])


epochs = 10
batch_size = 32

history = model.fit(X_train_, y_train, epochs=epochs, batch_size=batch_size,validation_split=0.2)

score,cat_acc = model.evaluate(X_test_,y_test)

y_pred = model.predict(X_test_)

print('loss : ', score)
print('categorical accuracy: ',cat_acc)

print('####################################')

thresholds = [0.01,0.04,0.06,0.08,0.1,0.12,0.14,0.16,0.2,0.25,0.26,0.27,0.28,0.29,0.3,0.35,0.4,0.5,0.6,0.7]
for val in thresholds:
    print("For threshold: ", val)
    pred=y_pred.copy()
  
    pred[pred>=val]=1
    pred[pred<val]=0
  
    precision = precision_score(y_test, pred, average='samples')
    recall = recall_score(y_test, pred, average='samples')
    f1 = f1_score(y_test, pred, average='samples')
   
    print("Samples-average quality numbers")
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

Train on 21059 samples, validate on 5265 samples
Epoch 1/10
21059/21059 [==============================] - 15s 693us/step - loss: 0.1118 - categorical_accuracy: 0.5384 - val_loss: 0.0176 - val_categorical_accuracy: 0.5597
Epoch 2/10
21059/21059 [==============================] - 13s 595us/step - loss: 0.0189 - categorical_accuracy: 0.5604 - val_loss: 0.0158 - val_categorical_accuracy: 0.5905
Epoch 3/10
21059/21059 [==============================] - 13s 612us/step - loss: 0.0155 - categorical_accuracy: 0.5836 - val_loss: 0.0150 - val_categorical_accuracy: 0.5928
Epoch 4/10
21059/21059 [==============================] - 10s 483us/step - loss: 0.0141 - categorical_accuracy: 0.5958 - val_loss: 0.0143 - val_categorical_accuracy: 0.6000
Epoch 5/10
21059/21059 [==============================] - 11s 543us/step - loss: 0.0131 - categorical_accuracy: 0.6110 - val_loss: 0.0136 - val_categorical_accuracy: 0.6160
Epoch 6/10
21059/21059 [==============================] - 14s 667us/step - loss: 0.012

/home/user1/DGS-env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Samples-average quality numbers
Precision: 0.5878, Recall: 0.7989, F1-measure: 0.6441
For threshold:  0.1
Samples-average quality numbers
Precision: 0.6065, Recall: 0.7834, F1-measure: 0.6544
For threshold:  0.12
Samples-average quality numbers
Precision: 0.6206, Recall: 0.7694, F1-measure: 0.6608
For threshold:  0.14
Samples-average quality numbers
Precision: 0.6306, Recall: 0.7564, F1-measure: 0.6640
For threshold:  0.16
Samples-average quality numbers
Precision: 0.6375, Recall: 0.7449, F1-measure: 0.6654
For threshold:  0.2
Samples-average quality numbers
Precision: 0.6485, Recall: 0.7260, F1-measure: 0.6664
For threshold:  0.25
Samples-average quality numbers
Precision: 0.6559, Recall: 0.7057, F1-measure: 0.6643
For threshold:  0.3
Samples-average quality numbers
Precision: 0.6561, Recall: 0.6853, F1-measure: 0.6570
For threshold:  0.35
Samples-average quality numbers
Precision: 0.6523, Recall: 0.6675, F1-measure: 0.6481
For threshold:  0.4
Samples-average quality numbers
Precision

## Commentaire : 
Les résultats sembles similaire. Toutefois, la loss de validation est bien meilleur sans pour autant augmenter le f1 score. Nous allons essayer de faire seulement un réseau basé sur l'attention. Nous utilison ici, une verion multiplicative de l'attention.

In [359]:
import keras
from keras_self_attention import SeqSelfAttention

inputs = keras.layers.Input(shape=(1,1000))
#lstm = keras.layers.Bidirectional(keras.layers.LSTM(units=256,
                                                    #return_sequences=True))(inputs)
att = SeqSelfAttention(attention_type=SeqSelfAttention.ATTENTION_TYPE_MUL,
                       kernel_regularizer=keras.regularizers.l2(1e-4),
                       bias_regularizer=keras.regularizers.l1(1e-4),
                       attention_regularizer_weight=1e-4,
                       name='Attention')(inputs)

flatten = keras.layers.Flatten(name='Flatten')(att)
dense = keras.layers.Dense(units=y_train.shape[1], name='Dense')(flatten)

model = keras.models.Model(inputs=inputs, outputs=[dense])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])


epochs = 20
batch_size = 32

history = model.fit(X_train_, y_train, epochs=epochs, batch_size=batch_size,validation_split=0.2)

score,cat_acc = model.evaluate(X_test_,y_test)

y_pred = model.predict(X_test_)

print('loss : ', score)
print('categorical accuracy: ',cat_acc)

print('####################################')

thresholds = [0.01,0.04,0.06,0.08,0.1,0.12,0.14,0.16,0.2,0.25,0.3,0.35,0.4,0.5,0.6,0.7]
for val in thresholds:
    print("For threshold: ", val)
    pred=y_pred.copy()
  
    pred[pred>=val]=1
    pred[pred<val]=0
  
    precision = precision_score(y_test, pred, average='samples')
    recall = recall_score(y_test, pred, average='samples')
    f1 = f1_score(y_test, pred, average='samples')
   
    print("Samples-average quality numbers")
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

Train on 21059 samples, validate on 5265 samples
Epoch 1/20
21059/21059 [==============================] - 18s 838us/step - loss: 0.0393 - categorical_accuracy: 0.3177 - val_loss: 0.0231 - val_categorical_accuracy: 0.4809
Epoch 2/20
21059/21059 [==============================] - 14s 687us/step - loss: 0.0262 - categorical_accuracy: 0.5653 - val_loss: 0.0212 - val_categorical_accuracy: 0.5742
Epoch 3/20
21059/21059 [==============================] - 15s 725us/step - loss: 0.0240 - categorical_accuracy: 0.5950 - val_loss: 0.0205 - val_categorical_accuracy: 0.5867
Epoch 4/20
21059/21059 [==============================] - 13s 626us/step - loss: 0.0228 - categorical_accuracy: 0.6030 - val_loss: 0.0200 - val_categorical_accuracy: 0.5909
Epoch 5/20
21059/21059 [==============================] - 14s 667us/step - loss: 0.0217 - categorical_accuracy: 0.6062 - val_loss: 0.0198 - val_categorical_accuracy: 0.5992
Epoch 6/20
21059/21059 [==============================] - 13s 626us/step - loss: 0.019

/home/user1/DGS-env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Samples-average quality numbers
Precision: 0.4750, Recall: 0.8283, F1-measure: 0.5668
For threshold:  0.06
Samples-average quality numbers
Precision: 0.5086, Recall: 0.8210, F1-measure: 0.5922
For threshold:  0.08
Samples-average quality numbers
Precision: 0.5357, Recall: 0.8101, F1-measure: 0.6108
For threshold:  0.1
Samples-average quality numbers
Precision: 0.5608, Recall: 0.8012, F1-measure: 0.6276
For threshold:  0.12
Samples-average quality numbers
Precision: 0.5825, Recall: 0.7924, F1-measure: 0.6414
For threshold:  0.14
Samples-average quality numbers
Precision: 0.5994, Recall: 0.7840, F1-measure: 0.6515
For threshold:  0.16
Samples-average quality numbers
Precision: 0.6130, Recall: 0.7728, F1-measure: 0.6577
For threshold:  0.2
Samples-average quality numbers
Precision: 0.6318, Recall: 0.7507, F1-measure: 0.6638
For threshold:  0.25
Samples-average quality numbers
Precision: 0.6495, Recall: 0.7273, F1-measure: 0.6678
For threshold:  0.3
Samples-average quality numbers
Precisio

### Commentaire : 

Le modèle semble réellement bloqué a une valeure seuil de 0.67. Que ce soit des réseau recurent ou bien des mécanismes d'attention le score reste identique après 10 epochs. Nous allons donc changer de représentation car nous pensons avoir atteint les limite de la représentation en tfidf+SVD.

## Et si l'on changait de représentation ? Utilisation d'Embedding globaux

In [362]:
from tensorflow.python.keras.preprocessing.text import Tokenizer
from tensorflow.python.keras.preprocessing.sequence import pad_sequences

In [363]:
df_train = train['DESCRIPTION_INCIDENT']+'. '+train['ETAT_PATIENT']#+ '. '+train['ACTION_PATIENT']
df_test = test['DESCRIPTION_INCIDENT']+'. '+test['ETAT_PATIENT']#+'. '+test['ACTION_PATIENT']

In [364]:
# The maximum number of words to be used. 
MAX_NB_WORDS = 100000
# Max number of words in each complaint.
MAX_SEQUENCE_LENGTH = 300
# This is fixed.
EMBEDDING_DIM = 300

tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(df_train.values)
word_index = tokenizer.word_index
print(len(word_index))

X_train = tokenizer.texts_to_sequences(df_train.values)
X_test = tokenizer.texts_to_sequences(df_test.values)
word2index_inputs =  tokenizer.word_index

X_train = pad_sequences(X_train,MAX_SEQUENCE_LENGTH)
X_test = pad_sequences(X_test,MAX_SEQUENCE_LENGTH)
print(X_train.shape,X_test.shape)

61213
(26324, 300) (6580, 300)


In [436]:
model = Sequential()
model.add(Embedding(MAX_NB_WORDS, EMBEDDING_DIM, input_length=X_train.shape[1]))
model.add(SpatialDropout1D(0.2))
model.add(LSTM(200))
model.add(Dense(y_train.shape[1],activation='softmax'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

epochs = 10
batch_size = 64

history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,validation_split=0.2)



y_pred = model.predict(X_test)

print('loss : ', score)
print('categorical accuracy: ',cat_acc)

print('####################################')

thresholds = [0.01,0.04,0.06,0.08,0.1,0.12,0.14,0.16,0.2,0.25,0.3,0.35,0.4,0.5,0.6,0.7]
for val in thresholds:
    print("For threshold: ", val)
    pred=y_pred.copy()
  
    pred[pred>=val]=1
    pred[pred<val]=0
  
    precision = precision_score(y_test, pred, average='samples')
    recall = recall_score(y_test, pred, average='samples')
    f1 = f1_score(y_test, pred, average='samples')
   
    print("Samples-average quality numbers")
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

/home/user1/DGS-env/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 21059 samples, validate on 5265 samples
Epoch 1/10
21059/21059 [==============================] - 641s 30ms/step - loss: 0.0197 - categorical_accuracy: 0.5669 - val_loss: 0.0154 - val_categorical_accuracy: 0.5932
Epoch 2/10
21059/21059 [==============================] - 620s 29ms/step - loss: 0.0170 - categorical_accuracy: 0.5896 - val_loss: 0.0149 - val_categorical_accuracy: 0.5953
Epoch 3/10
21059/21059 [==============================] - 722s 34ms/step - loss: 0.0164 - categorical_accuracy: 0.5963 - val_loss: 0.0148 - val_categorical_accuracy: 0.5947
Epoch 4/10
21059/21059 [==============================] - 696s 33ms/step - loss: 0.0160 - categorical_accuracy: 0.5967 - val_loss: 0.0146 - val_categorical_accuracy: 0.5888
Epoch 5/10
21059/21059 [==============================] - 751s 36ms/step - loss: 0.0157 - categorical_accuracy: 0.5984 - val_loss: 0.0146 - val_categorical_accuracy: 0.5793
Epoch 6/10
21059/21059 [==============================] - 766s 36ms/step - loss: 0.015

/home/user1/DGS-env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Samples-average quality numbers
Precision: 0.5695, Recall: 0.6641, F1-measure: 0.5887
For threshold:  0.12
Samples-average quality numbers
Precision: 0.5949, Recall: 0.6427, F1-measure: 0.5941
For threshold:  0.14
Samples-average quality numbers
Precision: 0.5973, Recall: 0.6260, F1-measure: 0.5883
For threshold:  0.16
Samples-average quality numbers
Precision: 0.5529, Recall: 0.5958, F1-measure: 0.5659
For threshold:  0.2
Samples-average quality numbers
Precision: 0.5599, Recall: 0.5831, F1-measure: 0.5654
For threshold:  0.25
Samples-average quality numbers
Precision: 0.5540, Recall: 0.5694, F1-measure: 0.5568
For threshold:  0.3
Samples-average quality numbers
Precision: 0.5498, Recall: 0.5525, F1-measure: 0.5480
For threshold:  0.35
Samples-average quality numbers
Precision: 0.5301, Recall: 0.5268, F1-measure: 0.5266
For threshold:  0.4
Samples-average quality numbers
Precision: 0.5084, Recall: 0.5036, F1-measure: 0.5046
For threshold:  0.5
Samples-average quality numbers
Precision

### Commentaire : 
Les embedding appris n'offre pas de meilleurs résultats pour l'instant. Nous allons essayer de créer des embeddings par colonnes de texte, cette solution c'etait avérée fructueuse dans le cas du tfidf.

## Et si l'on séparait les entrées ?

In [403]:
# Model constants.
EMBEDDING_DIM =300
MAX_SEQUENCE_LENGTH =300
MAX_NB_WORDS = 50000

def vectorize(df_train,df_test,MAX_NB_WORDS,MAX_SEQUENCE_LENGTH ):
    tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
    tokenizer.fit_on_texts(df_train.values)
    word_index = tokenizer.word_index
    print(len(word_index))

    X_train = tokenizer.texts_to_sequences(df_train.values)
    X_test = tokenizer.texts_to_sequences(df_test.values)
    word2index_inputs =  tokenizer.word_index

    X_train = pad_sequences(X_train,MAX_SEQUENCE_LENGTH)
    X_test = pad_sequences(X_test,MAX_SEQUENCE_LENGTH)
    return (X_train, X_test)
TRAIN = []
for col in ['DESCRIPTION_INCIDENT', 'ETAT_PATIENT', 'FABRICANT'] : 
    X_train,X_test = vectorize(train[col],test[col],MAX_NB_WORDS,MAX_SEQUENCE_LENGTH )
    TRAIN.append((X_train,X_test))

57288
21168
2262


In [437]:
inputs_1 = Input(shape=(MAX_SEQUENCE_LENGTH,))
inputs_2 = Input(shape=(MAX_SEQUENCE_LENGTH,))
inputs_3 = Input(shape=(MAX_SEQUENCE_LENGTH,))
#x = Embedding(input_dim=MAX_NB_WORDS, output_dim=EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH)(inputs)
#x = Reshape((MAX_SEQUENCE_LENGTH,1,))(inputs_1)
x = Embedding(input_dim=MAX_NB_WORDS, output_dim=EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH)(inputs_1)
x = SpatialDropout1D(0.2)(x)
x = LSTM(200)(x)


#y = Reshape((MAX_SEQUENCE_LENGTH,1,))(inputs_2)
y = Embedding(input_dim=MAX_NB_WORDS, output_dim=EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH)(inputs_2)
y = SpatialDropout1D(0.2)(y)
y = LSTM(200)(y)


#z = Reshape((MAX_SEQUENCE_LENGTH,1,))(inputs_3)
z = Embedding(input_dim=MAX_NB_WORDS, output_dim=EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH)(inputs_3)
z = SpatialDropout1D(0.2)(z)
z = LSTM(200)(z)


w = concatenate([x, y, z])

out =  Dense(y_train.shape[1],activation='softmax')(w)

model = keras.models.Model(inputs=[inputs_1,inputs_2,inputs_3], outputs=out)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['categorical_accuracy'])

model.summary()


Model: "model_14"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_67 (InputLayer)           (None, 300)          0                                            
__________________________________________________________________________________________________
input_68 (InputLayer)           (None, 300)          0                                            
__________________________________________________________________________________________________
input_69 (InputLayer)           (None, 300)          0                                            
__________________________________________________________________________________________________
embedding_15 (Embedding)        (None, 300, 300)     15000000    input_67[0][0]                   
___________________________________________________________________________________________

In [438]:
history = model.fit([TRAIN[0][0],TRAIN[1][0],TRAIN[2][0]], y_train, epochs=10, validation_split=0.2, verbose=1, batch_size = 64)

/home/user1/DGS-env/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 21059 samples, validate on 5265 samples
Epoch 1/10
21059/21059 [==============================] - 1123s 53ms/step - loss: 0.0183 - categorical_accuracy: 0.5735 - val_loss: 0.0139 - val_categorical_accuracy: 0.5920
Epoch 2/10
21059/21059 [==============================] - 1107s 53ms/step - loss: 0.0149 - categorical_accuracy: 0.6065 - val_loss: 0.0133 - val_categorical_accuracy: 0.6160
Epoch 3/10
21059/21059 [==============================] - 1043s 50ms/step - loss: 0.0135 - categorical_accuracy: 0.6428 - val_loss: 0.0131 - val_categorical_accuracy: 0.6161
Epoch 4/10
21059/21059 [==============================] - 1282s 61ms/step - loss: 0.0124 - categorical_accuracy: 0.6751 - val_loss: 0.0135 - val_categorical_accuracy: 0.5930
Epoch 5/10
21059/21059 [==============================] - 1119s 53ms/step - loss: 0.0110 - categorical_accuracy: 0.7212 - val_loss: 0.0139 - val_categorical_accuracy: 0.5899
Epoch 6/10
21059/21059 [==============================] - 1141s 54ms/step - loss:

In [439]:
y_pred = model.predict([TRAIN[0][1],TRAIN[1][1],TRAIN[2][1]])

print('loss : ', score)
print('categorical accuracy: ',cat_acc)

print('####################################')

thresholds = [0.01,0.04,0.06,0.08,0.1,0.12,0.14,0.16,0.2,0.25,0.3,0.35,0.4,0.5,0.6,0.7]
for val in thresholds:
    print("For threshold: ", val)
    pred=y_pred.copy()
  
    pred[pred>=val]=1
    pred[pred<val]=0
  
    precision = precision_score(y_test, pred, average='samples')
    recall = recall_score(y_test, pred, average='samples')
    f1 = f1_score(y_test, pred, average='samples')
   
    print("Samples-average quality numbers")
    print("Precision: {:.4f}, Recall: {:.4f}, F1-measure: {:.4f}".format(precision, recall, f1))

loss :  0.02118711931047831
categorical accuracy:  0.6486322283744812
####################################
For threshold:  0.01
Samples-average quality numbers
Precision: 0.4884, Recall: 0.7969, F1-measure: 0.5440
For threshold:  0.04
Samples-average quality numbers
Precision: 0.5703, Recall: 0.7271, F1-measure: 0.6039
For threshold:  0.06
Samples-average quality numbers
Precision: 0.5937, Recall: 0.7053, F1-measure: 0.6152
For threshold:  0.08


/home/user1/DGS-env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Samples-average quality numbers
Precision: 0.6082, Recall: 0.6877, F1-measure: 0.6195
For threshold:  0.1
Samples-average quality numbers
Precision: 0.6178, Recall: 0.6719, F1-measure: 0.6199
For threshold:  0.12
Samples-average quality numbers
Precision: 0.6259, Recall: 0.6592, F1-measure: 0.6192
For threshold:  0.14
Samples-average quality numbers
Precision: 0.6304, Recall: 0.6468, F1-measure: 0.6165
For threshold:  0.16
Samples-average quality numbers
Precision: 0.6310, Recall: 0.6369, F1-measure: 0.6134
For threshold:  0.2
Samples-average quality numbers
Precision: 0.6274, Recall: 0.6188, F1-measure: 0.6051
For threshold:  0.25
Samples-average quality numbers
Precision: 0.6157, Recall: 0.5973, F1-measure: 0.5920
For threshold:  0.3
Samples-average quality numbers
Precision: 0.5998, Recall: 0.5795, F1-measure: 0.5788
For threshold:  0.35
Samples-average quality numbers
Precision: 0.5893, Recall: 0.5662, F1-measure: 0.5696
For threshold:  0.4
Samples-average quality numbers
Precision

# Commentaire :
Nous observons que notre modèle sur apprend très rapidement et de manière importante. nous avons deux solutions classiques pour contrer cet effet : 
- Regularisation
- Drop Out

# Conclusion : 
 Suite aux diverses expériences que nousa vosn mener dans ce notebook, nos conclusions sont : 
- La tf-idf est une représentation riche qui est difficile à battre tant en termes de performances qu'en temps de calcul.
- Les modèles d'attention régularisés évitent mieux le sur-apprentissage que les modèles recurent avec du drop out
- séparer les collones pour encoder


Ce que nous devons essayer : 
- Multi head attention https://www.kaggle.com/fareise/multi-head-self-attention-for-text-classification, https://github.com/CyberZHG/keras-multi-head
- Hierarchical attention : https://paperswithcode.com/paper/hierarchical-attentional-hybrid-neural
- Concatenation des embedings et du tfidf
- chercher de nouvelles méthodes de régularisation pour les réseaux récurrents
- tester les CNN : https://www.kaggle.com/sanikamal/text-classification-with-python-and-keras
- ajouter une couche de positinal encoding : https://github.com/kaushalshetty/Positional-Encoding
- Librairie à essyaer rapidement :
    - text-classification-keras : https://pypi.org/project/text-classification-keras/
    - pytext :  https://github.com/facebookresearch/pytext

- approche avec des emmbedings déjà entrainés : 
    - https://adventuresinmachinelearning.com/word2vec-keras-tutorial/
    - https://medium.com/@ppasumarthi_69210/word-embeddings-in-keras-be6bb3092831
    